In [109]:
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
from bs4 import BeautifulSoup
import markdown
import string
import re
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

def remove_html_tags(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Function to remove Markdown
def remove_markdown(text):
    html = markdown.markdown(text)
    return BeautifulSoup(html, "html.parser").get_text()

def clean_row(row):
    return row.apply(lambda x: remove_markdown(remove_html_tags(x)) if isinstance(x, str) else x)

def clean_text(text):
    text = re.sub(r'-', ' ', text)
    text = re.sub(r'#', 'unique_column_heading', text)
    text = re.sub(r'None', 'unique_column_heading', text)
    text = re.sub(r'<strong>', '', text)
    text = re.sub(r'</strong>', ' ', text)
    text = re.sub(r'<em>', ' ', text)
    text = re.sub(r'</em>', ' ', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Replace spaces with dashes
    text = re.sub(r'\s+', '-', text)
    return text

def preprocess_df(df):
    df = df.astype(str)
    df = df.reset_index(drop=True)
    return df

def merge_columns(df: pd.DataFrame) -> pd.DataFrame:
    merged_columns = {}

    # Iterate through columns
    for col in df.columns:
        if col in merged_columns:
            merged_columns[col] = merged_columns[col].fillna('') + ' ' + df[col].fillna('')
        else:
            merged_columns[col] = df[col].fillna('')

    merged_df = pd.DataFrame(merged_columns)

    return merged_df


In [110]:
mhs_2016 = pd.read_csv("resources/2016.csv", index_col=False)
mhs_2016["year"] = 2016
mhs_2016.columns = [clean_text(col) for col in mhs_2016.columns]
mhs_2016 = mhs_2016.apply(clean_row, axis=1)
mhs_2016 = preprocess_df(mhs_2016)

column_mapping = {
    "are-you-self-employed": "self-employed",
    "how-many-employees-does-your-company-or-organization-have": "numb-employees",
    "is-your-employer-primarily-a-tech-companyorganization": "company-tech",
    "is-your-primary-role-within-your-company-related-to-techit": "role-tech",
    "does-your-employer-provide-mental-health-benefits-as-part-of-healthcare-coverage": "mh-benefits",
    "do-you-know-the-options-for-mental-health-care-available-under-your-employer-provided-coverage": "know-mh-options",
    "has-your-employer-ever-formally-discussed-mental-health-for-example-as-part-of-a-wellness-campaign-or-other-official-communication": "discussed-mh",
    "does-your-employer-offer-resources-to-learn-more-about-mental-health-concerns-and-options-for-seeking-help": "mh-resources",
    "is-your-anonymity-protected-if-you-choose-to-take-advantage-of-mental-health-or-substance-abuse-treatment-resources-provided-by-your-employer": "anonymity-protected",
    "if-a-mental-health-issue-prompted-you-to-request-a-medical-leave-from-work-asking-for-that-leave-would-be": "mh-leave",
    "do-you-think-that-discussing-a-mental-health-disorder-with-your-employer-would-have-negative-consequences": "mh-negative-consequences",
    "do-you-think-that-discussing-a-physical-health-issue-with-your-employer-would-have-negative-consequences": "ph-negative-consequences",
    "would-you-feel-comfortable-discussing-a-mental-health-disorder-with-your-coworkers": "comfortable-coworkers",
    "would-you-feel-comfortable-discussing-a-mental-health-disorder-with-your-direct-supervisors": "comfortable-supervisors",
    "do-you-feel-that-your-employer-takes-mental-health-as-seriously-as-physical-health": "mh-seriousness",
    "have-you-heard-of-or-observed-negative-consequences-for-co-workers-who-have-been-open-about-mental-health-issues-in-your-workplace": "observed-negative",
    "do-you-have-medical-coverage-private-insurance-or-state-prov ided-which-includes-treatment-of-mental-health-issues": "mh-coverage",
    "do-you-know-local-or-online-resources-to-seek-help-for-a-mental-health-disorder": "know-local-resources",
    "if-you-have-been-diagnosed-or-treated-for-a-mental-health-disorder-do-you-ever-reveal-this-to-clients-or-business-contacts": "reveal-clients",
    "if-you-have-revealed-a-mental-health-issue-to-a-client-or-business-contact-do-you-believe-this-has-impacted-you-negatively": "mh-impact-clients",
    "if-you-have-been-diagnosed-or-treated-for-a-mental-health-disorder-do-you-ever-reveal-this-to-coworkers-or-employees": "reveal-coworkers",
    "if-you-have-revealed-a-mental-health-issue-to-a-coworker-or-employee-do-you-believe-this-has-impacted-you-negatively": "mh-impact-coworkers",
    "do-you-believe-your-productivity-is-ever-affected-by-a-mental-health-issue": "affect-job",
    "if-yes-what-percentage-of-your-work-time-time-performing-primary-or-secondary-job-functions-is-affected-by-a-mental-health-issue": "percent-affect",
    "do-you-have-previous-employers": "previous-companies",
    "have-your-previous-employers-provided-mental-health-benefits": "mh-benefits",
    "were-you-aware-of-the-options-for-mental-health-care-provided-by-your-previous-employers": "aware-benefits",
    "did-your-previous-employers-ever-formally-discuss-mental-health-as-part-of-a-wellness-campaign-or-other-official-communication": "previous-discussed-mh",
    "did-your-previous-employers-provide-resources-to-learn-more-about-mental-health-issues-and-how-to-seek-help": "previous-mh-resources",
    "was-your-anonymity-protected-if-you-chose-to-take-advantage-of-mental-health-or-substance-abuse-treatment-resources-with-previous-employers": "previous-anonymity-protected",
    "do-you-think-that-discussing-a-mental-health-disorder-with-previous-employers-would-have-negative-consequences": "previous-mh-negative-consequences",
    "do-you-think-that-discussing-a-physical-health-issue-with-previous-employers-would-have-negative-consequences": "previous-ph-negative-consequences",
    "would-you-have-been-willing-to-discuss-a-mental-health-issue-with-your-previous-co-workers": "previous-comfortable-coworkers",
    "would-you-have-been-willing-to-discuss-a-mental-health-issue-with-your-direct-supervisors": "previous-comfortable-supervisors",
    "did-you-feel-that-your-previous-employers-took-mental-health-as-seriously-as-physical-health": "previous-mh-seriousness",
    "did-you-hear-of-or-observe-negative-consequences-for-co-workers-with-mental-health-issues-in-your-previous-workplaces": "previous-observed-negative",
    "would-you-be-willing-to-bring-up-a-physical-health-issue-with-a-potential-employer-in-an-interview": "tell-physical-issue",
    "why-or-why-not": "why-or-why-not-physical-issue",
    "would-you-bring-up-a-mental-health-issue-with-a-potential-employer-in-an-interview": "tell-mental-issue",
    "why-or-why-not1": "why-or-why-not-mental-issue",
    "do-you-feel-that-being-identified-as-a-person-with-a-mental-health-issue-would-hurt-your-career": "mh-hurt-career",
    "do-you-think-that-team-membersco-workers-would-view-you-more-negatively-if-they-knew-you-suffered-from-a-mental-health-issue": "mh-view-negatively",
    "how-willing-would-you-be-to-share-with-friends-and-family-that-you-have-a-mental-illness": "share-mh-family",
    "have-you-observed-or-experienced-an-unsupportive-or-badly-handled-response-to-a-mental-health-issue-in-your-current-or-previous-workplace": "observed-bad-response",
    "have-your-observations-of-how-another-individual-who-discussed-a-mental-health-disorder-made-you-less-likely-to-reveal-a-mental-health-issue-yourself-in-your-current-workplace": "less-likely-reveal",
    "do-you-have-a-family-history-of-mental-illness": "family-history-mh",
    "have-you-had-a-mental-health-disorder-in-the-past": "past-mh-disorder",
    "do-you-currently-have-a-mental-health-disorder": "current-mh-disorder",
    "if-yes-what-conditions-have-you-been-diagnosed-with": "conditions-diagnosed",
    "if-maybe-what-conditions-do-you-believe-you-have": "conditions-believe",
    "have-you-been-diagnosed-with-a-mental-health-condition-by-a-medical-professional": "been-diagnosed",
    "if-so-what-conditions-were-you-diagnosed-with": "been-diagnosed-with",
    "have-you-ever-sought-treatment-for-a-mental-health-issue-from-a-mental-health-professional": "sought-treatment",
    "interferes-with-your-work-when-being-treated-effectively": "interferes-when-treated",
    "if-you-have-a-mental-health-issue-do-you-feel-that-it-interferes-with-your-work-when-not-being-treated-effectively": "interferes-when-not-treated",
    "what-is-your-age": "age",
    "what-is-your-gender": "gender",
    "what-country-do-you-live-in": "live-country",
    "what-us-state-or-territory-do-you-live-in": "live-state",
    "what-country-do-you-work-in": "work-country",
    "what-us-state-or-territory-do-you-work-in": "work-state",
    "which-of-the-following-best-describes-your-work-position": "position",
    "do-you-work-remotely": "remote",
    "year": "year"
}
mhs_2016 = pd.read_csv("resources/2016.csv", index_col=False)
mhs_2016["year"] = 2016
mhs_2016.columns = [clean_text(col) for col in mhs_2016.columns]
mhs_2016 = mhs_2016.apply(clean_row, axis=1)
mhs_2016 = preprocess_df(mhs_2016)
mhs_2016.rename(columns=column_mapping, inplace=True)

column_names = mhs_2016.columns.tolist()

chunk_size = 20
for i in range(0, len(column_names), chunk_size):
    print(column_names[i:i+chunk_size])

/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/2876397856.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()
/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/2876397856.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


['uniquecolumnheading', 'self-employed', 'numb-employees', 'company-tech', 'role-tech', 'mh-benefits', 'know-mh-options', 'discussed-mh', 'mh-resources', 'anonymity-protected', 'mh-leave', 'mh-negative-consequences', 'ph-negative-consequences', 'comfortable-coworkers', 'comfortable-supervisors', 'mh-seriousness', 'observed-negative', 'do-you-have-medical-coverage-private-insurance-or-state-provided-which-includes-treatment-of-mental-health-issues', 'know-local-resources', 'reveal-clients']
['mh-impact-clients', 'reveal-coworkers', 'mh-impact-coworkers', 'affect-job', 'percent-affect', 'previous-companies', 'mh-benefits', 'aware-benefits', 'previous-discussed-mh', 'previous-mh-resources', 'previous-anonymity-protected', 'previous-mh-negative-consequences', 'previous-ph-negative-consequences', 'previous-comfortable-coworkers', 'previous-comfortable-supervisors', 'previous-mh-seriousness', 'previous-observed-negative', 'tell-physical-issue', 'why-or-why-not-physical-issue', 'tell-mental-i

In [111]:
new_column_mapping = {
    'are-you-self-employed-': 'self-employed',
    'how-many-employees-does-your-company-or-organization-have': 'numb-employees',
    'is-your-employer-primarily-a-tech-companyorganization': 'company-tech',
    'is-your-primary-role-within-your-company-related-to-techit': 'role-tech',
    'does-your-employer-provide-mental-health-benefits-as-part-of-healthcare-coverage': 'mh-benefits',
    'do-you-know-the-options-for-mental-health-care-available-under-your-employer-provided-health-coverage': 'know-mh-options',
    'has-your-employer-ever-formally-discussed-mental-health-for-example-as-part-of-a-wellness-campaign-or-other-official-communication': 'discussed-mh',
    'does-your-employer-offer-resources-to-learn-more-about-mental-health-disorders-and-options-for-seeking-help': 'mh-resources',
    'is-your-anonymity-protected-if-you-choose-to-take-advantage-of-mental-health-or-substance-abuse-treatment-resources-provided-by-your-employer': 'anonymity-protected',
    'if-a-mental-health-issue-prompted-you-to-request-a-medical-leave-from-work-how-easy-or-difficult-would-it-be-to-ask-for-that-leave': 'mh-leave',
    'would-you-feel-more-comfortable-talking-to-your-coworkers-about-your-physical-health-or-your-mental-health': 'comfortable-coworkers',
    'would-you-feel-comfortable-discussing-a-mental-health-issue-with-your-direct-supervisors': 'comfortable-supervisors',
    'have-you-ever-discussed-your-mental-health-with-your-employer': 'discussed-mh-employer',
    'describe-the-conversation-you-had-with-your-employer-about-your-mental-health-including-their-reactions-and-what-actions-were-taken-to-address-your-mental-health-issuequestions': 'describe-conv-employer',
    'would-you-feel-comfortable-discussing-a-mental-health-issue-with-your-coworkers': 'comfortable-coworkers',
    'have-you-ever-discussed-your-mental-health-with-coworkers': 'discussed-mh-coworkers',
    'describe-the-conversation-with-coworkers-you-had-about-your-mental-health-including-their-reactions': 'describe-conv-coworkers',
    'have-you-ever-had-a-coworker-discuss-their-or-another-coworkers-mental-health-with-you': 'coworker-discuss-mh',
    'describe-the-conversation-your-coworker-had-with-you-about-their-mental-health-please-do-not-use-names': 'describe-conv-coworker',
    'overall-how-much-importance-does-your-employer-place-on-physical-health': 'employer-physical-health',
    'overall-how-much-importance-does-your-employer-place-on-mental-health': 'employer-mental-health',
    'do-you-have-medical-coverage-private-insurance-or-state-provided-that-includes-treatment-of-mental-health-disorders': 'mh-coverage',
    'do-you-know-local-or-online-resources-to-seek-help-for-a-mental-health-issue': 'know-local-resources',
    'if-you-have-been-diagnosed-or-treated-for-a-mental-health-disorder-do-you-ever-reveal-this-to-clients-or-business-contacts-': 'reveal-clients',
    'if-you-have-revealed-a-mental-health-disorder-to-a-client-or-business-contact-how-has-this-affected-you-or-the-relationship': 'mh-impact-clients',
    'if-you-have-been-diagnosed-or-treated-for-a-mental-health-disorder-do-you-ever-reveal-this-to-coworkers-or-employees-': 'reveal-coworkers',
    'if-you-have-revealed-a-mental-health-disorder-to-a-coworker-or-employee-how-has-this-impacted-you-or-the-relationship': 'mh-impact-coworkers',
    'do-you-believe-your-productivity-is-ever-affected-by-a-mental-health-issue': 'affect-job',
    'if-yes-what-percentage-of-your-work-time-time-performing-primary-or-secondary-job-functions-is-affected-by-a-mental-health-issue': 'percent-affect',
    'do-you-have-previous-employers-': 'previous-companies',
    'was-your-employer-primarily-a-tech-companyorganization': 'previous-company-tech',
    'have-your-previous-employers-provided-mental-health-benefits-': 'prev-mh-benefits',
    'were-you-aware-of-the-options-for-mental-health-care-provided-by-your-previous-employers-': 'aware-prev-benefits',
    'did-your-previous-employers-ever-formally-discuss-mental-health-as-part-of-a-wellness-campaign-or-other-official-communication': 'previous-discussed-mh',
    'did-your-previous-employers-provide-resources-to-learn-more-about-mental-health-disorders-and-how-to-seek-help': 'previous-mh-resources',
    'was-your-anonymity-protected-if-you-chose-to-take-advantage-of-mental-health-or-substance-abuse-treatment-resources-with-previous-employers': 'previous-anonymity-protected',
    'would-you-have-felt-more-comfortable-talking-to-your-previous-employer-about-your-physical-health-or-your-mental-health': 'previous-comfortable-coworkers',
    'would-you-have-been-willing-to-discuss-your-mental-health-with-your-direct-supervisors': 'previous-comfortable-supervisors',
    'did-you-ever-discuss-your-mental-health-with-your-previous-employer': 'discussed-mh-employer',
    'describe-the-conversation-you-had-with-your-previous-employer-about-your-mental-health-including-their-reactions-and-actions-taken-to-address-your-mental-health-issuequestions': 'describe-conv-employer',
    'would-you-have-been-willing-to-discuss-your-mental-health-with-your-coworkers-at-previous-employers-': 'previous-comfortable-coworkers',
    'did-you-ever-discuss-your-mental-health-with-a-previous-coworkers': 'discussed-mh-coworkers',
    'describe-the-conversation-you-had-with-your-previous-coworkers-about-your-mental-health-including-their-reactions': 'describe-conv-coworkers',
    'did-you-ever-have-a-previous-coworker-discuss-their-or-another-coworkers-mental-health-with-you': 'coworker-discuss-mh',
    'describe-the-conversation-your-coworker-had-with-you-about-their-mental-health-please-do-not-use-names1': 'describe-conv-coworker',
    'overall-how-much-importance-did-your-previous-employer-place-on-physical-health': 'previous-employer-physical-health',
    'overall-how-much-importance-did-your-previous-employer-place-on-mental-health': 'previous-employer-mental-health',
    'do-you-currently-have-a-mental-health-disorder': 'current-mh-disorder',
    'have-you-ever-been-diagnosed-with-a-mental-health-disorder': 'been-diagnosed',
    'anxiety-disorder-generalized-social-phobia-etc': 'anxiety-disorder',
    'mood-disorder-depression-bipolar-disorder-etc': 'mood-disorder',
    'psychotic-disorder-schizophrenia-schizoaffective-etc': 'psychotic-disorder',
    'eating-disorder-anorexia-bulimia-etc': 'eating-disorder',
    'attention-deficit-hyperactivity-disorder': 'adhd',
    'personality-disorder-borderline-antisocial-paranoid-etc': 'personality-disorder',
    'obsessive-compulsive-disorder': 'ocd',
    'post-traumatic-stress-disorder': 'ptsd',
    'stress-response-syndromes': 'stress-syndromes',
    'dissociative-disorder': 'dissociative-disorder',
    'substance-use-disorder': 'substance-disorder',
    'addictive-disorder': 'addictive-disorder',
    'other': 'other',
    'anxiety-disorder-generalized-social-phobia-etc1': 'anxiety-disorder',
    'mood-disorder-depression-bipolar-disorder-etc1': 'mood-disorder',
    'psychotic-disorder-schizophrenia-schizoaffective-etc1': 'psychotic-disorder',
    'eating-disorder-anorexia-bulimia-etc1': 'eating-disorder',
    'attention-deficit-hyperactivity-disorder1': 'adhd',
    'personality-disorder-borderline-antisocial-paranoid-etc1': 'personality-disorder',
    'obsessive-compulsive-disorder1': 'ocd',
    'post-traumatic-stress-disorder1': 'ptsd',
    'stress-response-syndromes1': 'stress-syndromes',
    'dissociative-disorder1': 'dissociative-disorder',
    'substance-use-disorder1': 'substance-disorder',
    'addictive-disorder1': 'addictive-disorder',
    'other1': 'other',
    'anxiety-disorder-generalized-social-phobia-etc2': 'anxiety-disorder',
    'mood-disorder-depression-bipolar-disorder-etc2': 'mood-disorder',
    'psychotic-disorder-schizophrenia-schizoaffective-etc2': 'psychotic-disorder',
    'eating-disorder-anorexia-bulimia-etc2': 'eating-disorder',
    'attention-deficit-hyperactivity-disorder2': 'adhd',
    'personality-disorder-borderline-antisocial-paranoid-etc2': 'personality-disorder',
    'obsessive-compulsive-disorder2': 'ocd',
    'post-traumatic-stress-disorder2': 'ptsd',
    'stress-response-syndromes2': 'stress-syndromes',
    'dissociative-disorder2': 'dissociative-disorder',
    'substance-use-disorder2': 'substance-disorder',
    'addictive-disorder2': 'addictive-disorder',
    'other2': 'other',
    'have-you-had-a-mental-health-disorder-in-the-past': 'past-mh-disorder',
    'have-you-ever-sought-treatment-for-a-mental-health-disorder-from-a-mental-health-professional': 'sought-treatment',
    'do-you-have-a-family-history-of-mental-illness': 'family-history-mh',
    'if-you-have-a-mental-health-disorder-how-often-do-you-feel-that-it-interferes-with-your-work-when-being-treated-effectively-': 'interferes-when-treated',
    'if-you-have-a-mental-health-disorder-how-often-do-you-feel-that-it-interferes-with-your-work-when-not-being-treated-effectively-ie-when-you-are-experiencing-symptoms-': 'interferes-when-not-treated',
    'have-your-observations-of-how-another-individual-who-discussed-a-mental-health-issue-made-you-less-likely-to-reveal-a-mental-health-issue-yourself-in-your-current-workplace': 'less-likely-reveal',
    'how-willing-would-you-be-to-share-with-friends-and-family-that-you-have-a-mental-illness': 'share-mh-family',
    'would-you-be-willing-to-bring-up-a-physical-health-issue-with-a-potential-employer-in-an-interview': 'tell-physical-issue',
    'why-or-why-not': 'why-or-why-not-physical-issue',
    'would-you-bring-up-your-mental-health-with-a-potential-employer-in-an-interview': 'tell-mental-issue',
    'why-or-why-not1': 'why-or-why-not-mental-issue',
    'are-you-openly-identified-at-work-as-a-person-with-a-mental-health-issue': 'openly-identified',
    'has-being-identified-as-a-person-with-a-mental-health-issue-affected-your-career': 'mh-hurt-career',
    'how-has-it-affected-your-career': 'how-hurt-career',
    'if-they-knew-you-suffered-from-a-mental-health-disorder-how-do-you-think-that-team-membersco-workers-would-react': 'mh-view-negatively',
    'have-you-observed-or-experienced-an-unsupportive-or-badly-handled-response-to-a-mental-health-issue-in-your-current-or-previous-workplace-': 'observed-bad-response',
    'describe-the-circumstances-of-the-badly-handled-or-unsupportive-response': 'describe-bad-response',
    'have-you-observed-or-experienced-supportive-or-well-handled-response-to-a-mental-health-issue-in-your-current-or-previous-workplace-': 'observed-good-response',
    'describe-the-circumstances-of-the-supportive-or-well-handled-response': 'describe-good-response',
    'overall-how-well-do-you-think-the-tech-industry-supports-employees-with-mental-health-issues': 'tech-support-mh',
    'briefly-describe-what-you-think-the-industry-as-a-whole-andor-employers-could-do-to-improve-mental-health-support-for-employees': 'improve-mh-support',
    'if-there-is-anything-else-you-would-like-to-tell-us-that-has-not-been-covered-by-the-survey-questions-please-use-this-space-to-do-so': 'other-comments',
    'would-you-be-willing-to-talk-to-one-of-us-more-extensively-about-your-experiences-with-mental-health-issues-in-the-tech-industry-note-that-all-interview-responses-would-be-used-anonymously-and-only-with-your-permission': 'interview-willingness',
    'what-is-your-age': 'age',
    'what-is-your-gender': 'gender',
    'what-country-do-you-live-in': 'live-country',
    'what-us-state-or-territory-do-you-live-in': 'live-state',
    'what-is-your-race': 'race',
    'other3': 'other',
    'what-country-do-you-work-in': 'work-country',
    'what-us-state-or-territory-do-you-work-in': 'work-state',
    'start-date-utc': 'start-date-utc',
    'submit-date-utc': 'submit-date-utc',
    'network-id': 'network-id',
    'year': 'year'
}

mhs_2017 = pd.read_csv("resources/2017.csv", index_col=False)
mhs_2017["year"] = 2017
mhs_2017.columns = [clean_text(col) for col in mhs_2017.columns]
mhs_2017 = mhs_2017.apply(clean_row, axis=1)
mhs_2017 = preprocess_df(mhs_2017)
mhs_2017.rename(columns=new_column_mapping, inplace=True)

mhs_2017 = merge_columns(mhs_2017)

column_names = mhs_2017.columns.tolist()

chunk_size = 20
for i in range(0, len(column_names), chunk_size):
    print(column_names[i:i+chunk_size])

/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/2876397856.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


ValueError: 2

In [95]:
mhs_2018 = pd.read_csv("resources/2018.csv", index_col=False)
mhs_2018["year"] = 2018
mhs_2018.columns = [clean_text(col) for col in mhs_2018.columns]
mhs_2018 = mhs_2018.apply(clean_row, axis=1)
mhs_2018 = preprocess_df(mhs_2018)
mhs_2018.rename(columns=new_column_mapping, inplace=True)

column_names = mhs_2018.columns.tolist()

chunk_size = 20
for i in range(0, len(column_names), chunk_size):
    print(column_names[i:i+chunk_size])

/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/3757670727.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


['uniquecolumnheading', 'self-employed', 'numb-employees', 'company-tech', 'role-tech', 'mh-benefits', 'know-mh-options', 'discussed-mh', 'mh-resources', 'anonymity-protected', 'mh-leave', 'comfortable-coworkers', 'comfortable-supervisors', 'discussed-mh-employer', 'describe-conv-employer', 'comfortable-coworkers', 'discussed-mh-coworkers', 'describe-conv-coworkers', 'coworker-discuss-mh', 'describe-conv-coworker']
['employer-physical-health', 'employer-mental-health', 'mh-coverage', 'know-local-resources', 'reveal-clients', 'mh-impact-clients', 'reveal-coworkers', 'mh-impact-coworkers', 'affect-job', 'percent-affect', 'previous-companies', 'previous-company-tech', 'prev-mh-benefits', 'aware-prev-benefits', 'previous-discussed-mh', 'previous-mh-resources', 'previous-anonymity-protected', 'previous-comfortable-coworkers', 'previous-comfortable-supervisors', 'discussed-mh-employer']
['describe-conv-employer', 'previous-comfortable-coworkers', 'discussed-mh-coworkers', 'describe-conv-cowo

In [96]:
mhs_2019 = pd.read_csv("resources/2019.csv", index_col=False)
mhs_2019["year"] = 2019
mhs_2019.columns = [clean_text(col) for col in mhs_2019.columns]
mhs_2019 = mhs_2019.apply(clean_row, axis=1)
mhs_2019 = preprocess_df(mhs_2019)
mhs_2019.rename(columns=new_column_mapping, inplace=True)

column_names = mhs_2018.columns.tolist()

chunk_size = 20
for i in range(0, len(column_names), chunk_size):
    print(column_names[i:i+chunk_size])

/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/3757670727.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


['uniquecolumnheading', 'self-employed', 'numb-employees', 'company-tech', 'role-tech', 'mh-benefits', 'know-mh-options', 'discussed-mh', 'mh-resources', 'anonymity-protected', 'mh-leave', 'comfortable-coworkers', 'comfortable-supervisors', 'discussed-mh-employer', 'describe-conv-employer', 'comfortable-coworkers', 'discussed-mh-coworkers', 'describe-conv-coworkers', 'coworker-discuss-mh', 'describe-conv-coworker']
['employer-physical-health', 'employer-mental-health', 'mh-coverage', 'know-local-resources', 'reveal-clients', 'mh-impact-clients', 'reveal-coworkers', 'mh-impact-coworkers', 'affect-job', 'percent-affect', 'previous-companies', 'previous-company-tech', 'prev-mh-benefits', 'aware-prev-benefits', 'previous-discussed-mh', 'previous-mh-resources', 'previous-anonymity-protected', 'previous-comfortable-coworkers', 'previous-comfortable-supervisors', 'discussed-mh-employer']
['describe-conv-employer', 'previous-comfortable-coworkers', 'discussed-mh-coworkers', 'describe-conv-cowo

In [76]:


mhs_2020 = pd.read_csv("resources/2020.csv", index_col=False)
mhs_2020["year"] = 2020

mhs_2021 = pd.read_csv("resources/2021.csv", index_col=False)
mhs_2021["year"] = 2021

mhs_2022 = pd.read_csv("resources/2022.csv", index_col=False)
mhs_2022["year"] = 2022

mhs_2023 = pd.read_csv("resources/2023.csv", index_col=False)
mhs_2023["year"] = 2023






/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/855217152.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()
/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/855217152.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()
/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/855217152.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()
/var/folders/lp/71p77wmn3x7dxlg4mnwx61cw0000gq/T/ipykernel_35434/855217152.py:2: MarkupResemblesLocatorWarning: The input

In [79]:
mhs_2016.set_index("uniquecolumnheading", inplace=True)
mhs_2017.set_index("uniquecolumnheading", inplace=True)
mhs_2018.set_index("uniquecolumnheading", inplace=True)
mhs_2019.set_index("uniquecolumnheading", inplace=True)
mhs_2020.set_index("uniquecolumnheading", inplace=True)
mhs_2021.set_index("uniquecolumnheading", inplace=True)
mhs_2022.set_index("uniquecolumnheading", inplace=True)
mhs_2023.set_index("uniquecolumnheading", inplace=True)
mhs_concat=pd.concat([mhs_2016, mhs_2017,mhs_2018,mhs_2019,mhs_2020,mhs_2021,mhs_2022,mhs_2023], axis=1)
mhs_concat.to_csv("resources/mhs_concat.csv")
mhs_concat

KeyError: "None of ['uniquecolumnheading'] are in the columns"

In [ ]:
column_names = mhs_concat.columns.tolist()

chunk_size = 20
for i in range(0, len(column_names), chunk_size):
    print(column_names[i:i+chunk_size])

['are-you-self-employed', 'how-many-employees-does-your-company-or-organization-have', 'is-your-employer-primarily-a-tech-companyorganization', 'is-your-primary-role-within-your-company-related-to-techit', 'does-your-employer-provide-mental-health-benefits-as-part-of-healthcare-coverage', 'do-you-know-the-options-for-mental-health-care-available-under-your-employer-provided-coverage', 'has-your-employer-ever-formally-discussed-mental-health-for-example-as-part-of-a-wellness-campaign-or-other-official-communication', 'does-your-employer-offer-resources-to-learn-more-about-mental-health-concerns-and-options-for-seeking-help', 'is-your-anonymity-protected-if-you-choose-to-take-advantage-of-mental-health-or-substance-abuse-treatment-resources-provided-by-your-employer', 'if-a-mental-health-issue-prompted-you-to-request-a-medical-leave-from-work-asking-for-that-leave-would-be', 'do-you-think-that-discussing-a-mental-health-disorder-with-your-employer-would-have-negative-consequences', 'do-y